In [ ]:
import regex as re
import json
datafiles= {
  "E1" : [''],
  "E2" : ['a', 'b'],
  "E3" : [''],
  "E4" : ['']
}
datasets = []

for i,edition in enumerate(list(datafiles.keys())):
  dataset = ""
  for file in datafiles.get(edition):
    with open(f"./dataset/NF_{edition}{file}.txt", "r", encoding='utf-8') as fr:
      dataset += fr.read()
      fr.close()
  # for trash in re.findall(r"<h1.*>.*|<h2.*>.*|<h3.*>.*|<a.*>.*|<div.*>.*|<hr>", dataset):
  #   print(trash)
  datasets.append(re.sub(r"\n{2,}\s+(?=[a-z])", " ", re.sub(r"&.+;", "", dataset)))

In [ ]:
b_tag_dict = []
next_chars = 500
d = datasets[0]

# BUILD POSITIVE 
for match in re.finditer(r"((?<=<b>).+<\/b>)(.*(?<=<b>).+<\/b>)*", d):
  g1 = match.group(0)
  matched_b_tag = re.sub(r"</b>.*<b>|</b>"," ",g1).strip()
  end_of_b_tag = match.end()  
  surrounding_text_match = re.search(r"([^<]{1,"+str(next_chars)+r"})(?=<|$)", d[end_of_b_tag:end_of_b_tag+next_chars])
  surrounding_text = surrounding_text_match.group(0) if surrounding_text_match else ""

  short_def = re.sub(r"\s+", " ", surrounding_text).strip()
  if len(short_def) > 0:
    b_tag_dict.append([f"{matched_b_tag} {short_def}", matched_b_tag])


# BUILD NEGATIVE
for match in re.finditer(r"(\n\n[^<]{10,500})(?=\n|$|<)", d):
  g = match.group(0)
  matched_text = re.sub(r"\s+", " ", g).strip()
  b_tag_dict.append([matched_text, ""])

print(len(b_tag_dict))
with open("NF_E1_B.json", "w") as b_json:
  json.dump(b_tag_dict, b_json, indent=2, ensure_ascii=False)